In [1]:
import argparse
import json
import logging
import os
import random
import sys
from glob import glob
from os.path import join, dirname, basename, exists

import pandas as pd

sys.path.append('.')

from scorer.main import evaluate
from sentence_transformers import SentenceTransformer, util
from nltk.tokenize import sent_tokenize

random.seed(0)
ROOT_DIR = os.getcwd()

sbert = SentenceTransformer('paraphrase-distilroberta-base-v1')

/home/borisovai/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
INFO : Load pretrained SentenceTransformer: paraphrase-distilroberta-base-v1
INFO : Did not find folder paraphrase-distilroberta-base-v1
INFO : Try to download model from server: https://sbert.net/models/paraphrase-distilroberta-base-v1.zip
INFO : Load SentenceTransformer from folder: /home/borisovai/.cache/torch/sentence_transformers/sbert.net_models_paraphrase-distilroberta-base-v1
INFO : Use pytorch device: cuda


In [3]:
def load_vclaims(dir):
    vclaims_fp = glob(f'{dir}/*.json')
    vclaims_fp.sort()
    vclaims = {}
    vclaims_list = []
    for vclaim_fp in vclaims_fp:
        with open(vclaim_fp) as f:
            vclaim = json.load(f)
        vclaims[vclaim['vclaim_id']] = vclaim
        vclaims_list.append(vclaim)
    return vclaims, vclaims_list

In [5]:
vclaims, vclaims_list = load_vclaims("baselines/politifact-vclaims")
all_iclaims = pd.read_csv("baselines/v1/iclaims.queries", sep='\t', names=['iclaim_id', 'iclaim'])
wanted_iclaim_ids = pd.read_csv("baselines/v1/train.tsv", sep='\t', names=['iclaim_id', '0', 'vclaim_id', 'relevance'])
wanted_iclaim_ids = wanted_iclaim_ids.iclaim_id.tolist()

iclaims = []
for iclaim_id in wanted_iclaim_ids:
    iclaim = all_iclaims.iclaim[all_iclaims.iclaim_id == iclaim_id].iloc[0]
    iclaims.append((iclaim_id, iclaim))

# index = "2b-english"

# # options are title, vclaim, text
# scores = get_scores(iclaims, vclaims_list, index, search_keys=args.keys, size=args.size)
# ngram_baseline_fpath = join(ROOT_DIR,
#                                 'baselines/data/subtask_2b_bm25_english_baselines/v1/train.tsv')
# formatted_scores = format_scores(scores)
# with open(ngram_baseline_fpath, 'w') as f:
#     f.write(formatted_scores)
#     maps, mrr, precisions = evaluate(args.dev_file_path, ngram_baseline_fpath)
# logging.info(f"S-BERT Baseline for Subtask-{args.subtask}--{args.lang}")
# logging.info(f'All MAP scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. {maps}')
# logging.info(f'MRR score {mrr}')
# logging.info(f'All P scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. {precisions}')

In [55]:
iclaim_encodings = []
iclaim_encodings.append(sbert.encode(iclaims[0]))
iclaim_encodings.append(sbert.encode(iclaims[2]))



texts = [vclaim['text'] for vclaim in vclaims_list]
encodings_list = []

encodings_list.append(sbert.encode(sent_tokenize(texts[30])))
encodings_list.append(sbert.encode(sent_tokenize(texts[18])))
                           



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [75]:
iclaims_count, vclaims_count = len(iclaims), len(vclaims_list)
scores = {}

# Compute the encodings for all sentences in a text
texts = [vclaim['text'] for vclaim in vclaims_list]
# iclaims_encodings = [sbert.encode(iclaim) for iclaim in iclaims]
# vclaim_encodings = [sbert.encode(sent_tokenize(text)) for text in texts]
# logging.info(f"Geting RM5 scores for {iclaims_count} iclaims and {vclaims_count} vclaims")

print(iclaims[0][1])

# for iclaim_id, iclaim in iclaims:
    
#     score = get_score(iclaims_encodings[iclaims.index(iclaim_id, iclaim)], vclaims_list, vclaim_encodings, index, search_keys=search_keys, size=size)
#     scores[iclaim_id] = score
    
# print(scores)

You were very clear that you would not provide government assistance to the U.S. auto companies even if they went through bankruptcy.


In [17]:
print(encodings_list[0])
# print(sentences[33:34].shape)
# test_results = util.semantic_search(query, sentences[6:7])
# print(test_results)


[[ 0.13679329  0.34706703 -0.1844899  ...  0.5020161   0.06291218
  -0.18408445]
 [ 0.02565827  0.25984174  0.21223713 ... -0.24807751 -0.13124554
   0.03622852]
 [ 0.09518003  0.211623    0.00453631 ...  0.12868962 -0.14914145
   0.38807908]
 ...
 [ 0.36941403  0.45837355  0.06119161 ...  0.2959436  -0.01883373
   0.01633267]
 [ 0.07529093  0.36175445  0.08527803 ...  0.26322654 -0.03113398
  -0.16800277]
 [-0.01616848  0.4336011  -0.04188044 ...  0.30927145 -0.00233852
   0.14829345]]


In [48]:
score = {}
 
for i, encodding in enumerate(encodings_list):
        results = util.semantic_search(query, encodding, top_k=5)
        result_sum = 0
        for j, result in enumerate(results):
            result_sum += result[j]['score']
        average = result_sum / len(results)
        vclaim_id = vclaims_list[i]['vclaim_id']
        score[vclaim_id] = average
score = sorted(score.items(), key=lambda x: x[1], reverse=True)

print("Score:", score)

Score: [('vclaim-pol-00001', 0.22464996576309204), ('vclaim-pol-00000', 0.18482476472854614)]


In [52]:
score = {}
results = util.semantic_search(iclaim_encodings, encodings_list, top_k=5)

# for i, encodding in enumerate(encodings_list):
#         results = util.semantic_search(query, encodding, top_k=5)
#         result_sum = 0
#         for j, result in enumerate(results):
#             result_sum += result[j]['score']
#         average = result_sum / len(results)
#         vclaim_id = vclaims_list[i]['vclaim_id']
#         score[vclaim_id] = average
# score = sorted(score.items(), key=lambda x: x[1], reverse=True)

# print("Score:", score)



TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

In [ ]:
def get_scores(iclaims, vclaims_list, index, search_keys, size):
    iclaims_count, vclaims_count = len(iclaims), len(vclaims_list)
    scores = {}

    logging.info(f"Geting RM5 scores for {iclaims_count} iclaims and {vclaims_count} vclaims")

    for iclaim_id, iclaim in iclaims:
        score = get_score(iclaim, vclaims_list, index, search_keys=search_keys, size=size)
        scores[iclaim_id] = score
    return scores


def format_scores(scores):
    output_string = ''
    for iclaim_id in scores:
        for i, (vclaim_id, score) in enumerate(scores[iclaim_id]):
            output_string += f"{iclaim_id}\tQ0\t{vclaim_id}\t{i + 1}\t{score}\telasic\n"
    return output_string

In [ ]:

def run_baselines(args):
    if not exists('baselines/data'):
        os.mkdir('baselines/data')
    vclaims, vclaims_list = load_vclaims(args.vclaims_dir_path)
    all_iclaims = pd.read_csv(args.iclaims_file_path, sep='\t', names=['iclaim_id', 'iclaim'])
    wanted_iclaim_ids = pd.read_csv(args.dev_file_path, sep='\t', names=['iclaim_id', '0', 'vclaim_id', 'relevance'])
    wanted_iclaim_ids = wanted_iclaim_ids.iclaim_id.tolist()

    iclaims = []
    for iclaim_id in wanted_iclaim_ids:
        iclaim = all_iclaims.iclaim[all_iclaims.iclaim_id == iclaim_id].iloc[0]
        iclaims.append((iclaim_id, iclaim))

    index = f"{args.subtask}-{args.lang}"

    # options are title, vclaim, text
    scores = get_scores(iclaims, vclaims_list, index, search_keys=args.keys, size=args.size)
    ngram_baseline_fpath = join(ROOT_DIR,
                                f'baselines/data/subtask_{args.subtask}_bm25_{args.lang}_{basename(args.dev_file_path)}')
    formatted_scores = format_scores(scores)
    with open(ngram_baseline_fpath, 'w') as f:
        f.write(formatted_scores)
    maps, mrr, precisions = evaluate(args.dev_file_path, ngram_baseline_fpath)
    logging.info(f"S-BERT Baseline for Subtask-{args.subtask}--{args.lang}")
    logging.info(f'All MAP scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. {maps}')
    logging.info(f'MRR score {mrr}')
    logging.info(f'All P scores on threshold from [1, 3, 5, 10, 20, 50, 1000]. {precisions}')


# python baselines/bm25.py --train-file-path=baselines/v1/train.tsv --dev-file-path=baselines/v1/train.tsv --vclaims-dir-path=baselines/politifact-vclaims --iclaims-file-path=baselines/v1/iclaims.queries --subtask=2b --lang=english
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--train-file-path", "-t", required=True, type=str,
                        help="The absolute path to the training data")
    parser.add_argument("--dev-file-path", "-d", required=True, type=str,
                        help="The absolute path to the dev data")
    parser.add_argument("--vclaims-dir-path", "-v", required=True, type=str,
                        help="The absolute path to the directory with the verified claim documents")
    parser.add_argument("--iclaims-file-path", "-i", required=True,
                        help="TSV file with iclaims. Format: iclaim_id iclaim_content")
    parser.add_argument("--keys", "-k", default=['vclaim', 'title'],
                        help="Keys to search in the document")
    parser.add_argument("--size", "-s", default=19250,
                        help="Maximum results extracted for a query")
    parser.add_argument("--subtask", "-m", required=True,
                        choices=['2a', '2b'],
                        help="The subtask you want to check the format of.")
    parser.add_argument("--lang", "-l", required=True, type=str,
                        choices=['arabic', 'english'],
                        help="The language of the subtask")

    args = parser.parse_args()
    run_baselines(args)